# Linearity in Finance

> To-do list:
* Examining the CAPM model with the efficient frontier and capital market line
* Solving for the security market line using regression
* Examining the APT model and performing a multivariate linear regression
* Understanding linear optimization in portfolio allocation
* Linear optimization using the PuLP package
* Understanding the outcomes of linear programming
* Introduction to integer programming
* Implementing a linear integer programming model with binary conditions
* Solving systems of linear equations with equalities using matrix linear algebra
* Solving systems of linear equations directly with the LU, Cholesky, and QR decomposition
* Solving systems of linear equations indirectly with the Jacobi and Gauss-Seidel method

## CAPM model

$$ R_i = R_f+\beta_i(R_{mkt} - R_f) $$

The `scipy.stats.linregress` function returns the slope of the regression line, the intercept of the regression line, the correlation coefficient, the p-value for a hypothesis test with null hypothesis of a zero slope, and the standard error of the estimate

| Time period | Stock returns | Market returns |
| ----------- | ----------- | ----------- |
| 1 | 0.065 | 0.055 |
|2 |0.0265|-0.09
|3|-0.0593|-0.041
|4|-0.001|0.045
|5|0.0346|0.022






In [3]:
""" Linear regression with SciPy """
from scipy import stats
stock_returns = [0.065, 0.0265, -0.0593, -0.001, 0.0346]
mkt_returns = [0.055, -0.09, -0.041, 0.045, 0.022]
beta, alpha, r_value, p_value, std_err = stats.linregress(stock_returns, mkt_returns)

print(beta, alpha)

0.5077431878770808 -0.008481900352462384


* The equation describing the SML can be written as:
$$ E(R_i)=R_f +β_i [E(R_M)−R_f] $$

## The Arbitrage Pricing Theory (APT) model

$$ E[R_i]=α_i+β_{i,1}F_1+β_{i,2}F_2+...+β_{i,j}F_j $$

In [ ]:
""" Least squares regression with statsmodels """
import numpy as np
import statsmodels.api as sm
# Generate some sample data
num_periods = 9
all_values = np.array([np.random.random(8)
                        for i in range(num_periods)])
# Filter the data
y_values = all_values[:, 0]  # First column values as Y
x_values = all_values[:, 1:]  # All other values as X
x_values = sm.add_constant(x_values)  # Include the intercept
results = sm.OLS(y_values, x_values).fit()  # Regress and fit the model